# グラフ探索アルゴリズムI 第1回

## グラフ探索問題の表現方法

### グラフに関する用語

グラフとは点と線からなるデータ構造である。この講義では節点 (node) と辺 (edge) という。節点のことは vertex などと言うこともある。辺は branch や arc などとも言う。  
対象となる問題によっては違う呼び方をすることもある。例えばゲームやパズルでは、節点を局面 (position) 、辺を着手 (move) と呼ぶこともある。

<img src="fig1-1.png" width=200>  
グラフ1

### グラフの表現

グラフを表現するデータ構造は様々なものがある。
以下の例は図のグラフを隣接行列によって表現した物である。
小さなグラフ、あるいは密 (dense) なグラフは隣接行列で表現するのが自然である。
(密なグラフとは枝 (edge) の本数が多いグラフのことを指す。)
S, a, b, c, d, G をそれぞれ 0, 1, 2, 3, 4, 5 番目の要素としている。行列の要素が辺の長さを示す。長さ0で接続がないことを現している。1行目は、節点aは節点cへコスト4で接続されていることを示す。有向グラフなので、逆向きは接続がないので0が入っている。（なお、このグラフは UC Berkeley の講義 CS188 の宿題を引用したものである。）

In [3]:
import numpy as np

graph_array = np.array(
#     [S, a, b, c, d, G]
    [[ 0, 2, 3, 0, 5, 0], # S
     [ 0, 0, 0, 4, 0, 0], # a
     [ 0, 0, 0, 0, 4, 0], # b
     [ 0, 0, 0, 1, 0, 2], # c
     [ 0, 0, 0, 0, 0, 5], # d
     [ 0, 0, 0, 0, 0, 0]] # G
)

print(graph_array)

[[0 2 3 0 5 0]
 [0 0 0 4 0 0]
 [0 0 0 0 4 0]
 [0 0 0 1 0 2]
 [0 0 0 0 0 5]
 [0 0 0 0 0 0]]


隣接行列は分かりやすい方法だが、節点数 $N$ に対して $N^2$ のメモリを必要とする。
節点 (node) ごとの枝 (edge) が少ない疎 (sparse) なグラフの場合、隣接リストを用いるとメモリを大幅に節約可能である。

In [4]:
graph_list = [
              [(1, 2), (2, 3), (4, 5)], # S
              [(3, 4)],                 # a
              [(4, 4)],                 # b
              [(4, 1), (5, 2)],         # c
              [(5, 5)],                 # d
              []                        # G
]

問題に合わせてグラフのデータ構造を選択する必要がある。

## 探索問題の表現

探索問題の定義は、探索グラフ自体の定義に加えて探索の目的を含んだものと言える。以下はグラフを配列で表現した場合の最短経路問題の一例である。

In [5]:
class Problem1:
    def __init__(self, s, g, graph_array):
        self.start = s # node S
        self.goal = g # node G
        self.graph = graph_array

prob1 = Problem1(0, 5, graph_array)

## 基本的なグラフ探索アルゴリズム

### 幅優先探索 (Breadth-First Search, BFS) 

非常に単純なアルゴリズムである幅優先探索の動作を説明しよう。
幅優先探索は、スタートから1段ずつ全ての可能な経路を探索する単純なアルゴリズムである。
簡単なアルゴリズムではあるが、実装するためには準備が必要である。
まず節点はこの場合、どのようなデータ構造で表現するべきだろうか。
以下のコードを見て欲しい。節点を示すクラス Node には state, dist, parent の3つの変数を持たせている。それぞれ以下の意味がある。
- state: 節点を一意に特定可能なデータである必要がある。この場合には簡単で、節点に番号を振ってあるので一つの数字で良い。
- dist: 探索中に判明したスタートからの距離を保存する変数である。
- parent: この節点に到達する直前に訪問された節点である。これを覚えておくと最後に経路を表示する際に便利（直下のコードでは未使用）


In [6]:
class Node:
    def __init__(self, state, distance, parent_state):
        self.st = state
        self.dist = distance
        self.par = parent_state


    # 以下は、printで表示するための物なのでこのまま真似をしてください
    def __str__(self):
        return "id:" + str(self.st) + " dist=" + str(self.dist) + " par=" + str(self.par)
    
    def __repr__(self):
        return f'Node({self.st}, {self.dist}, {self.par})'


上で定義した Node を使って幅優先探索を行う python コードを以下に示す。
この探索アルゴリズムは、現在探索されている接点を frontier というリストに保存する。frontier には最初はスタート接点Sのみが入っている（20行目）。
frontier は FIFO (First-in Fisrt-out) となっている。
このアルゴリズムは、各ステップで、
- frontier の先頭の接点を取り出し、かつ除去する (popしている, FIFOの動作, 28行目)
- その節点を<span style="color:red">展開 (expand)</span>し、子節点のリストを得る (29行目, expand関数も参照)
- 子節点のリストを frontier に追加する (33行目)

という動作をする。
途中で子節点がゴール節点Gであったら（31〜32行）解を返して終了する。(menuバーの View->Show Line Numbersで行番号を表示できる。）


In [7]:
class BFS:
    def __init__(self, problem):
        self.prob = problem
        self.node_num = self.prob.graph.shape[0]
  
    def expand(self, node):
        # return list of child nodes
        child_nodes = []
        for c in range(0, self.node_num):
            d = self.prob.graph[node.st][c]
            if d != 0:
                child_nodes.append( Node(c, node.dist + d, node.st) )
        return child_nodes
    
    def search(self):
        # this is the start node
        # state = 0 (Start node id)
        # dist = 0 (0 distance from the start node)
        # parent state = -1 (means there is no parent for this node)
        start_node = Node(self.prob.start, 0, -1)

        if (start_node.st == self.prob.goal): # checking exceptional case, start == goal
            return start_node
    
        frontier = [start_node] # queue (FIFO) of the frontier nodes
        print("frontier nodes " + str(frontier))

        while len(frontier) != 0:
            n = frontier.pop(0)
            child_nodes = self.expand(n)
            for c in child_nodes:
                if c.st == self.prob.goal:
                    return c
            frontier = frontier + child_nodes
            print("frontier nodes " + str(frontier))


solver = BFS(prob1)
goal_node = solver.search()
print ("search finished. distance to goal is " + str(goal_node.dist))

frontier nodes [Node(0, 0, -1)]
frontier nodes [Node(1, 2, 0), Node(2, 3, 0), Node(4, 5, 0)]
frontier nodes [Node(2, 3, 0), Node(4, 5, 0), Node(3, 6, 1)]
frontier nodes [Node(4, 5, 0), Node(3, 6, 1), Node(4, 7, 2)]
search finished. distance to goal is 10


以上の幅優先探索で一応、ゴール節点Gまでの距離を計算することができた。
さて、このアルゴリズムにはいくつか問題がある。

まず、この答えは正しいだろうか？
念のために言うともちろん正しくない。
なぜ間違ったのか説明できるだろうか？

さらに動作に無駄がありそうである。
各ステップの frontier の内容を示すと以下の表のようになる。

| step | frontier |
|------|----------|
| 0    | S        |
| 1    | a b d    |
| 2    | b d c    |
| 3    | d c d    |
| 4    | c d G    |

各ステップで左端の節点を取り出し (pop) その子節点を右端に追加している (FIFOの動作)。
step 3 を見て欲しい。節点dが2個入っている。
これで良いのだろうか？

通常、探索アルゴリズムの性質を調べる上で、まず考えることは以下の4つである。

1. completeness (解があるなら出力できるか)
1. optimality (出力された解は最善か)
1. time complexity (求解にどの程度の時間がかかるか)
1. memory complexity (メモリ使用量はどの程度か, frontier は最大でどの程度の大きさになるか）

上で示したBFSはどうだろうか。
まず completeness は満たしているだろうか？今回は解を発見できたが、どんなグラフに対してもそうだろうか？
例えばこのグラフについても解を出力するだろうか？

<img src="fig1-2.png" width=200>  
グラフ2

ちょっと引っかけで申し訳ないが、実は解を出力することが可能である。
先ほどのBFSは、他の問題はあるにせよ、スタートからゴールへの経路が存在すればそのうちの1つを発見することができる。

#### 課題 1-1, Jupyter Notebook の実行と編集の練習

> graph_array の定義を変更してグラフ2を表現するようにせよ。
> その後、BFSを動作させ、出力を以下に記入せよ。

ここは各自自由に編集できる。以下に BFS の出力をペーストせよ。

---

---

In [8]:
graph_array2=np.array(
#     [S, a, b, c, d, G]
    [[ 0, 2, 3, 0, 5, 0], # S
     [ 2, 0, 0, 4, 0, 0], # a
     [ 0, 0, 0, 0, 4, 0], # b
     [ 0, 0, 0, 1, 0, 2], # c
     [ 0, 0, 0, 0, 0, 5], # d
     [ 0, 0, 0, 0, 0, 0]] # G
)
prob2 = Problem1(0, 5, graph_array)
solver = BFS(prob2)
goal_node = solver.search()
print ("search finished. distance to goal is " + str(goal_node.dist))

frontier nodes [Node(0, 0, -1)]
frontier nodes [Node(1, 2, 0), Node(2, 3, 0), Node(4, 5, 0)]
frontier nodes [Node(2, 3, 0), Node(4, 5, 0), Node(3, 6, 1)]
frontier nodes [Node(4, 5, 0), Node(3, 6, 1), Node(4, 7, 2)]
search finished. distance to goal is 10


次の optimality はもちろん満たさない。
最短経路は S->a->c->G を通る長さ8 (3 step) の経路だが、
このBFSは節点を1段ずつたどるため、より step数が少ない S->d->G の長さ10の経路を先に発見している。

Time complexity と Memory complexity はどちらも frontier のサイズが直接関係する。
BFSの場合、frontier のサイズはグラフの節点数と比較してどうだろうか。
例えば、全ての節点が $b$ 個の子節点を持つようなグラフを探索した場合、
frontier の大きさは $ 1 + b^2 + b^3 + ... $ となって、探索深さ $d$ の時点ではオーダーは $b^d$となる。
さらに、先ほど示したBFSは、冗長な経路（合流やサイクルなど）を無視しているため問題がさらに大きい。
先ほどの step 3 で節点 d が複数あったことを思い出して欲しい。
例えば 2D-grid で最短経路を探索する場合、過去に訪問した節点を何度も訪問すると frontier の大きさは非常に大きくなってしまう。

### 深さ優先探索 (Depth-First Search, DFS)

次に、少しBFSと性質の違うアルゴリズムを紹介する。
先ほどのBFSは全ての節点を1段展開していく動作をする。
なので探索<span style="color:red">深さ</span>3にあったS->a->c->Gの経路ではなくより浅い深さ2の S->d->G の経路を出力した。
1段ずつ展開するのではなく、まずそのままできるだけ深く潜るアルゴリズムを考える。
これを深さ優先探索という。

In [9]:
class DFS:
    def __init__(self, problem):
        self.prob = problem
        self.node_num = self.prob.graph.shape[0]
  
    def expand(self, node):
        # return list of child nodes
        child_nodes = []
        for c in range(0, self.node_num):
            d = self.prob.graph[node.st][c]
            if d != 0:
                child_nodes.append( Node(c, node.dist + d, node.st) )
        return child_nodes
    
    def search(self):
        # this is the start node
        # state = 0 (Start node id)
        # dist = 0 (0 distance from the start node)
        # parent state = -1 (means there is no parent for this node)
        start_node = Node(self.prob.start, 0, -1)

        if (start_node.st == self.prob.goal): # checking exceptional case, start == goal
            return start_node
    
        frontier = [start_node] # stack (LIFO) of the frontier nodes
        print("frontier nodes " + str(frontier))

        while len(frontier) != 0:
            n = frontier.pop(0)
            child_nodes = self.expand(n)
            for c in child_nodes:
                if c.st == self.prob.goal:
                    return c
            frontier = child_nodes + frontier
            print("frontier nodes " + str(frontier))


solver = DFS(prob1)
goal_node = solver.search()
print ("search finished. distance to goal is " + str(goal_node.dist))

frontier nodes [Node(0, 0, -1)]
frontier nodes [Node(1, 2, 0), Node(2, 3, 0), Node(4, 5, 0)]
frontier nodes [Node(3, 6, 1), Node(2, 3, 0), Node(4, 5, 0)]
search finished. distance to goal is 8


先ほどのBFSとは違う解が得られた。なぜだろうか。また、BFSと非常によく似ているが、どこが違うのだろうか？

DFSの探索順序はBFSと異なり、まず一直線にできるだけ深く探索を行う。
aが一番先に展開されていたため、S->a->cと進み、そこでcの子節点にGが含まれていたので最短経路が発見された。
BFSとは動作が全く異なるが、実はコードは33行目の1箇所しか変更されていない。
BFSの時はfrontierを queue (First-In First-Out) として処理したが、
DFSではfrontierが stack (Last-In First-Out) となっている。
これだけの違いである。

DFSの場合は以下の性質を満たしているだろうか。
1. completeness (解があるなら出力できるか)
1. optimality (出力された解は最善か)
1. time complexity (求解にどの程度の時間がかかるか)
1. memory complexity (メモリ使用量はどの程度か, frontier は最大でどの程度の大きさになるか）

このDFSは残念ながらグラフにサイクルがあると無限ループしてしまう。
また、今回は偶然最善解を発見したが、一般にはその保証は全く無い。
Time complexity は評価が難しいが、運が良ければ解を早く見つけることもある。
最後の点はどうだろうか。
実はこれがDFSの利点であって、frontierのサイズは探索深さ$d$対して比例する程度に押さえられる。
先ほどBFSのところで紹介した、全ての節点が$b$個の子節点を持つようなグラフを探索すると
深さ$d$の時点での frontier のサイズは $bd$ 個になる。


### 反復深化深さ優先探索 (Iterative Deepening DFS)

DFSの利点が不明確なように見えないだろうか。
メモリ使用量は小さいが、そもそも解を発見できる保証もなく、計算時間も不安定なのでは
何のために存在するのか。実はDFSは重要なテクニックなのだが、ここでは
一つ非常に簡単な工夫を紹介する。
それが反復深化という方法である。
DFSを実行する際に、まず最大深さ $d$ までで探索を諦めるようにする。
（以下のコードではその場合に None を返している。）
深さ $d$ で解が発見できなければ $d+1$ まで探索する。
このようにして徐々に最大深さを増やしていき、解が発見されたところで終了する。
この方法は memory complexity が非常に小さく、
またBFSと同等の解を発見することができる。

非常に無駄なことをしているように見えるかも知れないが、探索空間によってはそうでもない。
もし深さが1増える毎に節点の個数が $b$ 倍になる場合、
一番時間がかかるのは最後の1回だからだ。

#### ちょっと質問

以下のコード例は明示的な frontier のリストを持たない。
また、ループではなく iter 関数を再帰呼び出しすることで探索を実行している。
しかし上記では frontier のサイズは $bd$ 個と言う説明をした。
どこにこれが保存されているのか説明できるだろうか？

In [10]:
class ID_DFS:
    def __init__(self, problem):
        self.prob = problem
        self.node_num = self.prob.graph.shape[0]
  
    def expand(self, node):
        # return list of child nodes
        child_nodes = []
        for c in range(0, self.node_num):
            d = self.prob.graph[node.st][c]
            if d != 0:
                child_nodes.append( Node(c, node.dist + d, node.st) )
        return child_nodes

    def iter(self, node, depth, max_depth):
        if node.st == self.prob.goal:
            return node
        if depth >= max_depth:
            return None
        child_nodes = self.expand(node)
        print(child_nodes)
        ret = None
        for c in child_nodes:
            found = self.iter(c, depth+1, max_depth)
            if found != None:
                ret = found
        return ret
    
    def search(self):
        # this is the start node
        # state = 0 (Start node id)
        # dist = 0 (0 distance from the start node)
        # parent state = -1 (means there is no parent for this node)
        start_node = Node(self.prob.start, 0, -1)

        if (start_node.st == self.prob.goal): # checking exceptional case, start == goal
            return start_node

        max_depth = 0
        while True:
            print ("max_depth:" + str(max_depth))
            found = self.iter(start_node, 0, max_depth)
            if found != None:
                return found
            max_depth += 1

solver = ID_DFS(prob1)
goal_node = solver.search()
print ("search finished. distance to goal is " + str(goal_node.dist))



max_depth:0
max_depth:1
[Node(1, 2, 0), Node(2, 3, 0), Node(4, 5, 0)]
max_depth:2
[Node(1, 2, 0), Node(2, 3, 0), Node(4, 5, 0)]
[Node(3, 6, 1)]
[Node(4, 7, 2)]
[Node(5, 10, 4)]
search finished. distance to goal is 10


#### 課題1-2 リスト形式でBFSを実装（Pythonの復習をかねて）

---
以下のコードの expand 関数のコメント部分を実装し、リスト形式で表現されたグラフに対してBFSを実装せよ。
graph_list の定義は冒頭にある。（以下のセルを直接編集しても良い。）

または Python, C++ などの言語で同等のコードを実装し、ソースコードを提出せよ。

---

In [14]:
class Problem2:
    def __init__(self):
        self.start = 0 # node S
        self.goal = 5 # node G
        self.graph = graph_list

        
class BFS2:
    def __init__(self, problem):
        self.prob = problem
        self.node_num = len(self.prob.graph)
  
    def expand(self, problem, node):
        # return list of child nodes
        child_nodes = []
        for c in problem.graph[node.st]:
            child_nodes.append(Node(c[0],node.dist+c[1],node.st))
        return child_nodes
    
    def search(self):
        # this is the start node
        # state = 0 (Start node id)
        # dist = 0 (0 distance from the start node)
        # parent state = -1 (means there is no parent for this node)
        start_node = Node(self.prob.start, 0, -1)

        if (start_node.st == self.prob.goal): # checking exceptional case, start == goal
            return start_node
    
        frontier = [start_node] # queue (FIFO) of the frontier nodes

        while len(frontier) != 0:
            n = frontier.pop(0)
            child_nodes = self.expand(self.prob, n)
            for c in child_nodes:
                if c.st == self.prob.goal:
                    return c
            frontier = frontier + child_nodes
            print("frontier nodes " + str(frontier))

prob2 = Problem2()
bfs = BFS2(prob2)
goal_node = bfs.search()
print ("search finished. distance to goal is " + str(goal_node.dist))

frontier nodes [Node(1, 2, 0), Node(2, 3, 0), Node(4, 5, 0)]
frontier nodes [Node(2, 3, 0), Node(4, 5, 0), Node(3, 6, 1)]
frontier nodes [Node(4, 5, 0), Node(3, 6, 1), Node(4, 7, 2)]
search finished. distance to goal is 10



ここまでの教訓はいくつかある。

- BFSは complete (解が存在するなら発見する) が、メモリ使用量が大きい
- DFSはサイクルがあるなどすると解を発見できないので complete ではない。メモリ使用量は少ない。
- Iterative Deepening DFS はメモリ使用量は少ないが、BFSと同等の解を発見する能力がある。

未解決なのは冗長な経路（合流）への対処である。
これがないと同じ節点を何回も探索する（同じ節点を再展開する）
ことになって時間もメモリも大変に無駄にしてしまう。
皆さんご存じの Dijkstra 法や"A*"探索は無駄な再展開をしない。
詳しくは次回以降説明する。

念のため、2D gridで再展開がどれだけ無駄か動作例を見てみよう。
（デモ）

## 状態空間について

グラフ探索では節点のことを状態と考えて、
探索される範囲全体を状態空間 (state space) や探索空間 (search space) と呼ぶことがある。
例えば先ほどから題材にしていた小さいグラフの状態空間は6個の節点からなり、
状態空間の大きさは6である、などと言う。
探索アルゴリズムの効率を考えるなら、6回節点を探索したら解を得られるとありがたい。
アルゴリズムの多くはその性質を満たしている。
そこで、状態空間の大きさを見積もることでグラフ探索問題の難しさをある程度、見積もることができる。


### 節点の表現方法について

先ほどグラフの例では6個であった。よって Node の定義の中の state は $ 0...5 $ の数字で現せば良かった。
少し問題が複雑になると状態の表現方法も少し考える必要がある。
例えば 8-puzzle の状態空間はどのようなデータ構造で表現できるだろうか。
また状態空間の大きさはどの程度だろうか。

<img src="fig1-3.png" width=400>  

以下はあくまで一例だが、9個の数字を用いた以下のような定義が考えられる。
図の一番上の盤面を示した例である。0が空白の場所を示す。

```python
[1, 2, 3, 4, 6, 0, 7, 5, 8]
```

状態空間の数は $ 9! $ と思うかも知れないが、実は偶奇性により
絶対に完成できない状態が半分存在するので $ 9! / 2 = 181440 $ 個である。


#### 課題 1-3 ハノイの塔

円盤が4個の場合のハノイの塔について考える。（ハノイの塔のルールが分からない場合は wikipedia などで調べよ。）
- 節点の状態をどのようなデータ構造で表現するべきか、案を示せ。
- ハノイの塔（円盤4枚）の場合、状態空間の大きさはどの程度か

<img src="fig1-4.png" width=300>  

回答欄に記入せよ。

以下、課題1-3回答欄

参考文献
- "Artificial Intelligence: A Modern Approach, 4th Global ed.", by Stuart Russell and Peter Norvig  
   http://aima.cs.berkeley.edu/index.html
- "ヒューリスティック探索入門", 陣内 佑  
   https://jinnaiyuu.github.io/pdf/textbook.pdf

In [18]:
# 3 stacks(start,middle,end) 
# start : stack_start=[4,3,2,1]
# target : stack_end=[4,3,2,1]
class hannoi_stack:
    def __init__(self,stack):
        self.stack=stack
        self.last=stack[-1]
    
    def limit_append(self,num):
        if num>self.last:
            return False
        else:
            self.stack.append(num)
            self.last=num
            return True
        
# 状態空間の大きさ : 3+3*2*(4+6)+6*6=99